In [4]:
!pip install tensorboardX

   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---------------------------------------- 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/426.9 kB ? eta -:--:--
   --------------------------------------- 426.9/426.9 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 5.27.2 which is incompatible.


In [5]:
import torch
import logging
from datetime import datetime
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim import AdamW
from tensorboardX import SummaryWriter

In [40]:
import sys
sys.path.append('C:\\VPTR_jigsaws\\utils')
sys.path.append('C:\\VPTR_jigsaws\\model')

In [41]:
from utils import dataset

In [52]:
from model import VPTREnc, VPTRDec, VPTRDisc, init_weights, VPTRFormerFAR
from model import GDL, MSELoss, GANLoss
from utils import get_dataloader, save_ckpt, load_ckpt, set_seed, AverageMeters, init_loss_dict, write_summary, resume_training
from utils import visualize_batch_clips
from utils import get_dataloader, VidToTensor, VidRandomHorizontalFlip, VidRandomVerticalFlip
from model import VPTREnc, VPTRDec, VPTRFormerFAR, MSELoss, GDL
from dataset import JIGSAWSDataset
from pathlib import Path
from model import VPTREnc, VPTRDec, VPTRDisc, init_weights, VPTRFormerFAR

In [53]:
# Setup directories and parameters
ckpt_save_dir = Path('C:\\VPTR_jigsaws\\jigsaws_suturing\\VPTR_ckpts\\JIGSAWS_FAR_ckpt')
tensorboard_save_dir = Path('C:\\VPTR_jigsaws\\jigsaws_suturing\\VPTR_ckpts\\JIGSAWS_FAR_tensorboard')
resume_AE_ckpt = Path('C:\\VPTR_jigsaws\\jigsaws_suturing\\VPTR_ckpts\\JIGSAWS_ResNetAE_MSEGDLgan_ckpt').joinpath('epoch_100.tar')
#resume_ckpt = ckpt_save_dir.joinpath('epoch_100.tar')
resume_ckpt = None

In [54]:
# Logging setup
logging.basicConfig(level=logging.INFO,
                    datefmt='%a, %d %b %Y %H:%M:%S',
                    format='%(asctime)s - %(message)s',
                    filename=f'{ckpt_save_dir}/train_log.log',
                    filemode='a')

In [55]:
start_epoch = 0
summary_writer = SummaryWriter(tensorboard_save_dir.absolute().as_posix())
num_past_frames = 10
num_future_frames = 20
encH, encW, encC = 8, 8, 528
img_channels = 3
epochs = 100
N = 4
#AE_lr = 2e-4
Transformer_lr = 1e-4
max_grad_norm = 1.0
rpe = False
lam_gan = 0.001
dropout = 0.1
device = torch.device('cuda:0')

In [56]:
##################### Init Dataset ###########################
data_set_name = 'Suturing'
dataset_dir = 'C:\\VPTR_jigsaws\\jigsaws_suturing\\frames_split\\'
train_loader, test_loader, renorm_transform = get_dataloader(data_set_name, N, dataset_dir, num_past_frames=num_past_frames, num_future_frames=num_future_frames)

In [57]:
# Model initialization
VPTR_Enc = VPTREnc(img_channels, feat_dim=encC, n_downsampling=3).to(device)
VPTR_Dec = VPTRDec(img_channels, feat_dim=encC, n_downsampling=3, out_layer='Sigmoid').to(device)
VPTR_Transformer = VPTRFormerFAR(num_past_frames, num_future_frames, encH=encH, encW=encW, d_model=encC, nhead=8, num_encoder_layers=12, dropout=dropout, window_size=4, Spatial_FFN_hidden_ratio=4, rpe=rpe).to(device)

In [58]:
# Optimizers
optimizer_T = AdamW(params=VPTR_Transformer.parameters(), lr=Transformer_lr)

In [59]:
# Load checkpoints if available
loss_name_list = ['T_MSE', 'T_GDL', 'T_gan', 'T_total', 'Dtotal', 'Dfake', 'Dreal']
loss_dict = init_loss_dict(loss_name_list)
mse_loss = MSELoss()
gdl_loss = GDL(alpha=1)

In [60]:
loss_dict, start_epoch = resume_training({'VPTR_Enc': VPTR_Enc, 'VPTR_Dec': VPTR_Dec}, {}, resume_AE_ckpt, loss_name_list)

In [61]:
if resume_ckpt:
    loss_dict, start_epoch = resume_training({'VPTR_Transformer': VPTR_Transformer}, {'optimizer_T': optimizer_T}, resume_ckpt, loss_name_list)

In [62]:
def single_iter(VPTR_Enc, VPTR_Dec, VPTR_Disc, VPTR_Transformer, optimizer_T, optimizer_D, sample, device, lam_gan, train_flag=True):
    past_frames, future_frames = sample
    past_frames = past_frames.to(device)
    future_frames = future_frames.to(device)
    
    with torch.no_grad():
        x = torch.cat([past_frames, future_frames[:, :-1, ...]], dim=1) #concatenate past frames and future frames except the last frame
        gt_feats = VPTR_Enc(x) #encode the concatenated frames
        
    if train_flag:
        VPTR_Transformer=VPTR_Transformer.train()
        VPTR_Transformer.zero_grad(set_to_none=True)
        VPTR_Dec.zero_grad(set_to_none=True)
        
        pred_future_feats = VPTR_Transformer(gt_feats) #predict the future features
        pred_frames = VPTR_Dec(pred_future_feats) #decode the predicted future features
        
        if optimizer_D is not None:
            assert lam_gan is not None, "Please input lam_gan"
            #update discriminator
            VPTR_Disc=VPTR_Disc.train()
            for p in VPTR_Disc.parameters():
                p.requires_grad_(True)
            VPTR_Disc.zero_grad(set_to_none=True)
            loss_D, loss_D_fake, loss_D_real = cal_lossD(VPTR_Disc, pred_frames, future_frames, lam_gan)
            loss_D.backward()
            optimizer_D.step()
        
            for p in VPTR_Disc.parameters():
                p.requires_grad_(False)


        pred_frames_resized = resize_tensor(pred_frames, (future_frames.shape[-2], future_frames.shape[-1]))
        
        ##update transformer(generator)
        loss_T, T_GDL_loss, T_MSE_loss, loss_T_gan = cal_lossT(pred_frames_resized, torch.cat([past_frames[:, 1:, ...], future_frames], dim=1), VPTR_Disc, lam_gan)
        loss_T.backward()
        nn.utils.clip_grad_norm_(VPTR_Transformer.parameters(), max_grad_norm, norm_type=2)
        optimizer_T.step()

    else:
        if optimizer_D is not None:
            VPTR_Disc=VPTR_Disc.eval()
        VPTR_Transformer= VPTR_Transformer.eval()
        with torch.no_grad():
            pred_future_feats = VPTR_Transformer(gt_feats)
            pred_frames = VPTR_Dec(pred_future_feats)
            if optimizer_D is not None:
                loss_D, loss_D_fake, loss_D_real = cal_lossD(VPTR_Disc, pred_frames, future_frames, lam_gan)
            else:
                loss_D, loss_D_fake, loss_D_real = torch.zeros(1), torch.zeros(1), torch.zeros(1)
                
                
            pred_frames_resized = resize_tensor(pred_frames, (future_frames.shape[-2], future_frames.shape[-1]))
            loss_T, T_GDL_loss, T_MSE_loss, loss_T_gan = cal_lossT(pred_frames, torch.cat([past_frames[:, 1:, ...], future_frames], dim=1), VPTR_Disc, lam_gan)
    
    if optimizer_D is None:        
        loss_D, loss_D_fake, loss_D_real = torch.zeros(1), torch.zeros(1), torch.zeros(1)

    iter_loss_dict = {'T_total': loss_T.item(), 'T_MSE': T_MSE_loss.item(), 'T_GDL': T_GDL_loss.item(), 'T_gan': loss_T_gan.item(), 'Dtotal': loss_D.item(), 'Dfake': loss_D_fake.item(), 'Dreal': loss_D_real.item()}
    
    return iter_loss_dic

In [63]:
# Training loop
for epoch in range(start_epoch + 1, start_epoch + epochs + 1):
    epoch_st = datetime.now()
    EpochAveMeter = AverageMeters(loss_name_list)
    for idx, sample in enumerate(train_loader, 0):
        iter_loss_dict = single_iter(VPTR_Enc, VPTR_Dec, VPTR_Disc, VPTR_Transformer, optimizer_T, optimizer_D, sample, device, lam_gan, train_flag=True)
        EpochAveMeter.iter_update(iter_loss_dict)

    loss_dict = EpochAveMeter.epoch_update(loss_dict, epoch, train_flag=True)
    write_summary(summary_writer, loss_dict, train_flag=True)
    FAR_show_sample(VPTR_Enc, VPTR_Dec, VPTR_Transformer, num_future_frames, sample, f'{ckpt_save_dir}/train_gifs_epoch{epoch}', test_phase=False)

    if epoch % val_per_epochs == 0:
        EpochAveMeter = AverageMeters(loss_name_list)
        for idx, sample in enumerate(val_loader, 0):
            iter_loss_dict = single_iter(VPTR_Enc, VPTR_Dec, VPTR_Disc, VPTR_Transformer, optimizer_T, optimizer_D, sample, device, lam_gan, train_flag=False)
            EpochAveMeter.iter_update(iter_loss_dict)
        loss_dict = EpochAveMeter.epoch_update(loss_dict, epoch, train_flag=False)
        write_summary(summary_writer, loss_dict, train_flag=False)
        
        for idx, sample in enumerate(test_loader, random.randint(0, len(test_loader) - 1)):
            FAR_show_sample(VPTR_Enc, VPTR_Dec, VPTR_Transformer, num_future_frames, sample, f'{ckpt_save_dir}/val_gifs_epoch{epoch}', test_phase=True)

        torch.save({'epoch': epoch, 'VPTR_Transformer': VPTR_Transformer.state_dict(), 'optimizer_T': optimizer_T.state.dict()}, f'{ckpt_save_dir}/epoch_{epoch}.tar')
    
    logging.info(f'epoch: {epoch}, epoch time: {datetime.now() - epoch_st}')

NameError: name 'VPTR_Disc' is not defined